In [11]:
import geopandas as gp
import pandas as pd
import numpy as np
from scipy.stats import linregress
import subprocess
import os,sys,re
import emcee
import statsmodels.api as sm
import pylab
from scipy.stats import norm,shapiro
import seaborn as sn
import corner
from scipy import optimize,linalg
import matplotlib.pyplot as plt
import multiprocessing 

def shell(cmd):
    #print(cmd)
    process = subprocess.Popen([sys.executable]+cmd.split(" "),
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,text=True)
    stdout, stderr = process.communicate()
    if not process.returncode==0:
        try:
            print(cmd)
            print(stdout)
            print(stderr)
        except:
            print("failed to decode stdout/stderr")
        return False
    print(stdout)
    return stdout



In [12]:
import os
def test_drape(input_net_filename,input_terrain_filename,spatial_mismatch_prior_scale,slope_prior_scale,slope_continuity_prior_scale,pitch_angle_prior_scale,maxiter,disable_autodiff,extra):
    bayes_output_filename = "../scratch/"+os.path.basename(input_net_filename)\
                            +"--"+os.path.basename(input_terrain_filename)\
                            +f"--bdrape{spatial_mismatch_prior_scale}.shp"
    disable_autodiff = "--DISABLE-AUTODIFF" if disable_autodiff else ""
    res=shell(f"../BayesianDrape.py --TERRAIN-INPUT={input_terrain_filename} --POLYLINE-INPUT={input_net_filename} "
          f"--OUTPUT={bayes_output_filename} --SPATIAL-MISMATCH-PRIOR-SCALE={spatial_mismatch_prior_scale} "
          f"--SLOPE-CONTINUITY-PRIOR-SCALE={slope_continuity_prior_scale} --SLOPE-PRIOR-SCALE={slope_prior_scale} "
          f"--PITCH-ANGLE-PRIOR-SCALE={pitch_angle_prior_scale} --SPATIAL-MISMATCH-MAX={spatial_mismatch_prior_scale*2} "
          f"--MAXITER={maxiter} {extra} --IGNORE-PROJ-MISMATCH --ITERATION-REPORT-EVERY=0 "
          f"{disable_autodiff}")
    if not res:
        return False
    else:
        wanted_from_stdout = 'Num params,Total time,Optimizer terminated with status,First step change in log likelihood,First step mean change in param,Final step change in log likelihood,Final step mean change in param,Total change in log likelihood,Total mean change in param'.split(",")
        convergence_stats = {}
        for item in wanted_from_stdout:
            m = re.search(f"{item}: ([^\n]*)",res)
            if m:
                convergence_stats[item]=(m.group(1))
            else:
                print(f"no match for {item}")
        m = re.search("Final optimizer log likelihood after ([0-9]*) iterations",res)
        convergence_stats["iterations"]=int(m.group(1))

    return bayes_output_filename,convergence_stats


In [15]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def elev_change(geom):
    ec = 0
    for (_,_,z1),(_,_,z2) in pairwise(list(geom.coords)):
        ec += abs(z2-z1)
    return ec

def get_resids_r_ll(df,x,y):
    lr = linregress(df[x],df[y])
    resids = df[y]-(lr.intercept+lr.slope*df[x])
    scale = resids.std()
    loglik = norm(loc=0,scale=scale).logpdf(resids).sum()
    return resids,lr.rvalue,loglik,lr.slope,lr.intercept
   
def evaluate_drape_raw(label,net_name,terrain_name,smps,disable_autodiff,extra=""):
    sps = 2.66
    scps = 90
    paps = 1.28
    maxiter=100000
    
    draped_net_filename,convergence_stats = test_drape(net_name,terrain_name,smps,sps,scps,paps,maxiter,disable_autodiff,extra)
    
    draped_net = gp.read_file(draped_net_filename)
    draped_net["draped_elev_change"]=[elev_change(geom) for geom in draped_net.geometry]
    draped_net["length"]=draped_net.geometry.length

    def height(s):
        if type(s)==float:
            return(s)
        if s[-1]=="m":
            s = s[0:-1]
        return float(s)
    
    possible_os_elevchange_cols = [("elevationG","elevatio_1"),("inDirectio","inOpposite")]
    ec_cols = None
    for ecpair in possible_os_elevchange_cols:
        if all([x in draped_net.columns for x in ecpair]):
            ec_cols = ecpair
            
    draped_net["os_elev_change"] = draped_net.apply(lambda row: height(row[ec_cols[0]])+height(row[ec_cols[1]]),
                                                    axis=1)

    draped_net["os_slope"] = draped_net.os_elev_change/draped_net.length
    draped_net["draped_slope"] = draped_net.draped_elev_change/draped_net.length
    
    if len(draped_net)>=3:
        resids,r,_,slope,intercept = get_resids_r_ll(draped_net,"draped_elev_change","os_elev_change")
        stat, p = shapiro(resids)
    else:
        resids = draped_net.os_elev_change-draped_net.draped_elev_change
        stat = p = r = slope = intercept = "na"
        
    draped_net["resids"]=resids
    draped_net.to_file(draped_net_filename)
    
    elev_error_per_len = abs(resids).sum()/draped_net.length.sum()
    
    outputs = [
        ("model",label),
        ("smp",smps),
        ("status",convergence_stats["Optimizer terminated with status"]),
        ("init_dll",float(convergence_stats["First step change in log likelihood"])),
        ("final_dll",float(convergence_stats["Final step change in log likelihood"])),
        ("init_dparam",float(convergence_stats["First step mean change in param"])),
        ("final_dparam",float(convergence_stats["Final step mean change in param"])),
        ("dll",float(convergence_stats["Total change in log likelihood"])),
        ("dparam",float(convergence_stats["Total mean change in param"])),
        ("iter",convergence_stats["iterations"]),
        ("R",r),
        ("coeff",slope),
        ("int",intercept),
        ("ec/len err",elev_error_per_len),
        ("min outlier",min(resids)),
        ("max outlier",max(resids)),
        ("Shapiro",stat),
        ("Shapiro_p (high means gaussian)",p),
        ("Time",convergence_stats["Total time"]),
        ("Nparams",convergence_stats["Num params"]),
        ("Nlinks",len(draped_net))
    ]
    for name,value in outputs:
        print(f"{name}: {value}")
    print()
    
    df_filename="multi_terrain_test.csv"
    try:
        df = pd.read_csv(df_filename)
    except FileNotFoundError:
        df = pd.DataFrame(columns = [x[0] for x in outputs])
    df = df.append(dict(outputs),ignore_index=True)
    df.to_csv(df_filename,index=False)
    
# loop over smp=0, smp=(recommended for cell size), smp=0/newz formulation
# for each terrain model

evaluate_drape_raw("crossval a470b 50m bayes","../data/a470_test_prep.shp","../data/wider_terrain_50/all.tif",25,False,"--DECOUPLE-FIELD=bridge")
evaluate_drape_raw("crossval a470b 50m drape","../data/a470_test_prep.shp","../data/wider_terrain_50/all.tif",0,False)

#evaluate_drape_raw("wye 50m bayes","../data/highways_finaltest_prep.shp","../data/all_os50_terrain.tif",25,False,"--DECOUPLE-FIELD=bridge")
#evaluate_drape_raw("wye 50m drape","../data/highways_finaltest_prep.shp","../data/all_os50_terrain.tif",0,False,"--DECOUPLE-FIELD=bridge")

##evaluate_drape_raw("all 50m drape","../Download_2120558/itn_local_reliable.shp","../data/wider_terrain_50/all.tif",0,False)

#evaluate_drape_raw("wye 5m bayes","../data/highways_finaltest_prep.shp","../data/terrain_5/terrain.tif",2.5,False,"--DECOUPLE-FIELD=bridge")
#evaluate_drape_raw("wye 5m drape","../data/highways_finaltest_prep.shp","../data/terrain_5/terrain.tif",0,False)

#evaluate_drape_raw("crossval 50m bayes","../data/validation_prep.shp","../data/wider_terrain_50/all.tif",25,False)
#evaluate_drape_raw("crossval 50m drape","../data/validation_prep.shp","../data/wider_terrain_50/all.tif",0,False)

#evaluate_drape_raw("flattish 50m bayes","../data/flattish_test_prep.shp","../data/wider_terrain_50/all.tif",25,False)
#evaluate_drape_raw("flattish 50m drape","../data/flattish_test_prep.shp","../data/wider_terrain_50/all.tif",0,False)



#evaluate_drape_raw("cross slope/cliff medium","road_cross_cliff_slope_medium.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("cross slope/cliff longish","road_cross_cliff_slope_longish.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("cross slope/cliff","road_cross_cliff_slope.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("cross slope/cliff short","road_cross_cliff_slope_short.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("cross slope/cliff very short","road_cross_cliff_slope_very_short.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("contour slope","road_contour_slope.shp","lores_test_terrain.tif",1000,True)
#evaluate_drape_raw("contour cliff","road_contour_cliff.shp","lores_test_terrain.tif",1000,True)




Using OSGB 1936 / British National Grid
Terrain height range from -340282346638528859811704183484516925440.00 to 1345.40
Inserting extra vertices on polylines to interpolate terrain
  (6784 extra vertices added)
Building spatial model
19795 points in total
19625 points for estimation
170 points are decoupled
0 points are fixed
Minimum estimated segment length 0.01
Mean estimated segment length: 13.16
Spatial mismatch prior scale is 25.0
Slope prior scale of 2.66° gives grade of 4.6%
Num params: 39420
Starting optimizer log likelihood = -50636.4
   (Offset likelihood 0.0, Slope likelihood -21542.0, Pitch angle likelihood -29094.4)

Optimizer terminated with status: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Total time: 166 seconds
Final optimizer log likelihood after 3781 iterations = -40401.2
   (Offset likelihood -3.3, Slope likelihood -20550.0, Pitch angle likelihood -19847.9)
First step change in log likelihood: 2.2132918886782136
First step mean change in param: 1.465875959253

<ipython-input-15-09d2ccce94af>:60: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  draped_net.to_file(draped_net_filename)


model: crossval a470b 50m bayes
smp: 25
status: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
init_dll: 2.2132918886782136
final_dll: 7.5228308560326695e-06
init_dparam: 1.4658759592531311e-05
final_dparam: 4.792836559183695e-10
dll: 10235.28055468708
dparam: 0.19996513516111125
iter: 3781
R: 0.9591182641094784
coeff: 0.8421084345040492
int: 0.37691726098319034
ec/len err: 0.011082907470148283
min outlier: -79.47290588021238
max outlier: 41.59879040681261
Shapiro: 0.4537951350212097
Shapiro_p (high means gaussian): 0.0
Time: 166 seconds
Nparams: 39420
Nlinks: 1712

Using OSGB 1936 / British National Grid
Maximum spatial mismatch set from larger terrain tile dimension: 50.00
Terrain height range from -340282346638528859811704183484516925440.00 to 1345.40
Inserting extra vertices on polylines to interpolate terrain
  (6784 extra vertices added)
Building spatial model
19795 points in total
19795 points for estimation
0 points are decoupled
0 points are fixed
Minimum estimated segment le